# **homework2_2023100674_이서은**

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os
import numpy as np
import pandas as pd
# torch: 딥러닝의 핵심 라이브러리
# torch.nn: 신경망 레이어(Layer)와 손실 함수(Loss Function) 정의
# torch.optim: 최적화 알고리즘(Optimizer) 정의 (예: Adam, SGD)
# torch.utils.data.DataLoader: 데이터를 미니배치(Mini-Batch) 단위로 불러오는 유틸리티 (학습 효율화 및 GPU 활용 목적 )
# wandb: 머신러닝 실험 추적 및 시각화 도구 (Weights & Biases)
# argparse: 스크립트 실행 시 인자(Argument)를 쉽게 처리하기 위한 모듈 (예: 배치 크기, 에폭 수 설정)

###  [요구사항 1]	titanic 딥러닝 모델 기본 훈련

_01_code/_08_learning_and_optimization/c_my_model_training_with_argparse_wandb.py 코드를	그대로 활용하되 titanic 데이터에 맞게 수정하여	코딩

 Wandb로 훈련 과정 데이터 올려 그래프	얻어 내기
 
 • Training	loss
 
 • Validation	loss
 
 링크 : https://wandb.ai/seoeun2866-/Titanic_ML_Competition

In [2]:
# 경로 설정 및 데이터셋 모듈 임포트
# titanic_dataset.py에서 정의된 전처리 함수 및 데이터셋 클래스를 임포트하고, 사용자 환경에 맞게 BASE_PATH를 설정합니다.

from pathlib import Path
BASE_PATH = "/Users/lg/git/link_dl" # 데이터셋 파일이 위치한 기본 경로를 사용자 환경에 맞게 지정
print("BASE_PATH:", BASE_PATH) # 출력 > BASE_PATH: /Users/lg/git/link_dl

import sys
sys.path.append(BASE_PATH) # BASE_PATH를 파이썬 모듈 검색 경로에 추가하여 데이터셋 모듈을 임포트 가능하게 함

from _03_homeworks.homework_2.titanic_dataset \
  import get_preprocessed_dataset, TitanicTestDataset # 전처리된 데이터셋을 가져오는 함수 및 테스트 데이터셋 클래스 임포트

BASE_PATH: /Users/lg/git/link_dl


In [3]:
# 데이터 로딩 및 DataLoader 생성
# 전처리된 데이터셋을 불러와 학습(Train), 검증(Validation), 테스트(Test)용 DataLoader 객체를 생성
    
def get_data():
  # 전처리 함수 호출: train, validation, test 3개의 dataset을 반환
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset() 
  
  # 데이터셋 크기 출력 (전처리 과정에서 수행된 분할 결과)
  print(f"Train size: {len(train_dataset)}, Validation size: {len(validation_dataset)}, Test size: {len(test_dataset)}") # 출력(main에서) > Train size: 713, Validation size: 178, Test size: 418

  # DataLoader 생성 (미니배치 경사 하강법 구현)
  # Mini-Batch GD는 속도와 안정성의 균형을 이루며 GPU 연산에 최적화되어 가장 널리 사용됨
  
  # 훈련 데이터 DataLoader:
  # batch_size=wandb.config.batch_size: wandb 설정(argparse)에서 가져온 배치 크기 사용. 여기서는 128이 최적으로 선택됨.
  # shuffle=True: 학습 시 데이터 순서를 섞어 모델의 일반화 성능을 높임 (경사 하강 경로의 진동을 추가하여 지역 최솟값(Local Minima)을 벗어나는데 도움을 줌) 
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    
  # 검증 및 테스트 DataLoader: 일반적으로 전체 데이터를 하나의 배치로 사용하여 정확한 손실 및 성능 측정
  # shuffle=False: 순서가 중요하지 않으므로 셔플하지 않음
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset)) 
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)
  
  return train_data_loader, validation_data_loader, test_data_loader # test_data_loader도 반환하도록 수정

In [4]:
# c_my_model_training_with_argparse_wandb.py 파일에서
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()
    # 요구사항 2를 위해 Activation Function을 변수로 설정 (Sigmoid vs. ReLU vs. ELU vs. Leaky ReLU)
    # [요구사항 3] 요구사항 2에서 살펴본 가장 좋은 성능을 보이는 'Sigmoid'로 모델 구성
    activation = nn.Sigmoid() 

    self.model = nn.Sequential(
      # 첫 번째 은닉층: n_input (10) -> n_hidden_unit_list[0] (20)
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      activation, 
      # 두 번째 은닉층: n_hidden_unit_list[0] (20) -> n_hidden_unit_list[1] (20)
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      activation,
      # 출력층: n_hidden_unit_list[1] (20) -> n_output (2, 생존/사망)
      # 이진 분류를 위한 다중 클래스 분류(2개 클래스) 형태로 출력
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

### [요구사항 2] Activation	Function 변경 및	선택

1.첫 번째와 두 번째 차트(wandb.pdf 파일에 있음)를 분석

1.1. 훈련 손실 (Training Loss) 분석

Leaky ReLU, ELU, 및 ReLU는 약 400 Epoch 이후부터 비슷한 낮은 손실(≈ 0.3 ~ 0.4)을 보인다. Sigmoid 역시 낮은 손실(≈ 0.3 ~ 0.4)을 달성하지만, 다른 ReLU 계열 함수들보다 수렴 속도가 약간 느려 보인다.훈련 데이터에 대해서는 모든 모델이 잘 학습했다.

1.2. 검증 손실 (Validation Loss) 분석

Sigmoid (빨간색 선)가 가장 낮은 검증 손실을 보이며, Epoch 200 이후부터 약 0.4 수준을 유지한다. 이는 일반화 성능이 가장 좋음을 시사한다. ReLU (녹색 선)는 약 0.45 부근에서 상대적으로 안정적이지만, Sigmoid보다 높다. ELU (보라색 선)와 Leaky ReLU (분홍색 선)는 Sigmoid와 ReLU보다 높은 검증 손실(≈ 0.48~ 0.55)을 보이며, 특히 Leaky ReLU는 변동성이 높다.

1.3. 결론: 최적의 활성화 함수

Sigmoid 활성화 함수를 사용한 모델이 훈련 및 검증 손실 측면에서 가장 좋은 성능을 보여준다. 훈련 손실은 낮게 유지하면서 검증 손실 또한 다른 모델들보다 낮고 안정적이다. 따라서 여기서는 Sigmoid 를 선택했다.

In [5]:
def get_model_and_optimizer():
  # Titanic 데이터셋의 입력 특징 개수: 10
  # Titanic 생존 여부 예측: 이진 분류 (0: 사망, 1: 생존) -> 클래스 개수: 2
  linear_model = MyModel(n_input = 10, n_output = 2)
    
  # 최적화 함수: Adam 사용, 학습률(learning_rate)은 wandb config에서 가져옴
  # Adam Optimizer란?
  # Adam (Adaptive Moment Estimation)은 오늘날 가장 널리 사용되고 성능이 좋은 최적화 알고리즘 중 하나이다.
  # 역할: 파라미터 업데이트
  # 1. Adam은 경사 하강법(Gradient Descent)의 한 종류이다.
  # 2. 적응적 학습률 (Adaptive Learning Rate): Adam은 각 파라미터(가중치 하나하나)마다 고유한 학습률을 가진다.
  # 기울기가 자주 변하거나 큰 파라미터는 학습률을 작게 조정하고, 변화가 적은 파라미터는 학습률을 크게 유지하여, 전체적으로 빠르고 안정적인 수렴을 돕는다.
  # 3. 모멘텀(Momentum) 활용: Adam은 과거의 기울기 정보(Momentum)를 이용하여 현재의 업데이트 방향을 결정한다. 
  # 이는 최적화 경로에서 발생할 수 있는 굴곡이나 평평한 구간(Plateau)을 더 잘 극복하고, 수렴 속도를 높이는 데 도움을 준다.
  # linear_model.parameters(): 최적화할 대상, 즉 신경망 모델의 모든 학습 가능한 파라미터(W와 b)를 Adam에게 알려준다.
  # lr=wandb.config.learning_rate: Adam의 기본 학습률(gamma)을 설정한다. Adam은 이 기본 학습률을 바탕으로 각 파라미터에 대한 적응적인 학습률을 내부적으로 계산한다.
  optimizer = optim.Adam(linear_model.parameters(), lr=wandb.config.learning_rate)

  return linear_model, optimizer

In [9]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  # 손실 함수: 이진 분류를 위한 Cross Entropy Loss 사용
  # 이 함수는 다중 클래스 분류(K개 클래스)의 NLL(Negative Log-Likelihood) 손실을 최소화하도록 유도하며
  # 내부적으로 모델의 출력 로짓(logits)에 Softmax를 적용하여 확률 분포(λ_i,k)를 얻은 후,
  # Negative Log-Likelihood Loss를 계산하는 과정이 결합되어 있음 (즉, Softmax + NLLLoss)
  loss_fn = nn.CrossEntropyLoss()
  next_print_epoch = 100
    
  for epoch in range(1, n_epochs + 1):
    # --- Training Phase ---
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target'] # 정답 레이블 (y_i)
      
      output_train = model(input) # 순전파(Forward Pass): NN(x_i; θ) 계산
      loss = loss_fn(output_train, target) # 손실 계산: Cross Entropy Loss L(θ)
      
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad() # 이전 단계에서 계산된 기울기 초기화
      loss.backward() # 역전파: 현재 손실에 대한 가중치의 기울기 계산
      optimizer.step() # 옵티마이저를 이용해 가중치 업데이트

    # --- Validation Phase ---
    loss_validation = 0.0
    num_validations = 0

    with torch.no_grad(): # 검증 단계에서는 기울기를 계산하고 저장할 필요가 없으므로 메모리 및 시간 절약
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']
        
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        
        loss_validation += loss.item()
        num_validations += 1
    
    # Wandb 로깅 : 훈련 및 검증 손실을 시각화 도구(Wandb)로 전송
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations,
    })

    # 100 Epoch마다 콘솔 출력
    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}, "
      ) 
      # 출력
      # Epoch 100, Training loss 0.5019, Validation loss 0.5021,
      # Epoch 200, Training loss 0.4227, Validation loss 0.4604, 
      # Epoch 300, Training loss 0.4001, Validation loss 0.4452,
      # Epoch 400, Training loss 0.3806, Validation loss 0.4214,
      next_print_epoch += 100


In [7]:
def generate_submission_csv(model, test_data_loader, base_path):
    # 테스트 데이터에 대한 예측을 수행하고 submission.csv를 생성
    model.eval()  # 모델을 평가 모드로 전환
    
    predictions = []
    
    with torch.no_grad():
        for test_batch in test_data_loader:
            input_test = test_batch['input']
            
            # 모델 예측 (logits)
            output_test = model(input_test)
            
            # 클래스 예측 (가장 높은 로짓 값에 해당하는 인덱스)
            # 이진 분류 (클래스 0 또는 1)에서, 로짓 값이 높은 쪽이 예측 클래스가 됨.
            # 이는 확률 분포 Pr(y|φ)에서 최대값을 갖는 y를 선택하는 과정과 동일함
            _, predicted_classes = torch.max(output_test, dim=1)
            
            # 예측 결과를 리스트에 추가
            predictions.extend(predicted_classes.numpy())
            
    # 원본 test.csv 로드 경로 추정 및 PassengerId 추출
    test_csv_path = os.path.join(base_path, "_03_homeworks", "homework_2", "test.csv")
    
    try:
        test_original_df = pd.read_csv(test_csv_path)
    except FileNotFoundError:
        print(f"Error: Could not find original test.csv at {test_csv_path}. Please check the path.")
        return

    # test.csv에 포함된 PassengerId 추출
    passenger_ids = test_original_df['PassengerId'].values
    
    if len(passenger_ids) != len(predictions):
        print(f"Error: PassengerId count ({len(passenger_ids)}) does not match prediction count ({len(predictions)}). Submission failed.")
        return

    # submission DataFrame 생성
    submission_df = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Survived': predictions # 예측된 생존 여부 (0 또는 1)
    })
    
    # submission.csv 파일 생성 (현재 작업 디렉토리에 저장) (Kaggle 제출 형식)
    submission_path = "submission.csv" 
    submission_df.to_csv(submission_path, index=False)
    
    print(f"submission.csv 파일이 생성되었습니다: {os.path.abspath(submission_path)}") # 출력(main 에서) > submission.csv 파일이 생성되었습니다: C:\Users\lg\git\link_dl\_04_your_code\submission.csv
    print("\n--- submission.csv head ---")
    print(submission_df.head())
    # 출력 (main 에서)
    #       PassengerId  Survived
    #    0          892         0
    #    1          893         0
    #    2          894         0
    #    3          895         0
    #    4          896         0
    print("---------------------------\n")

### [요구사항 3] 테스트 및 submission.csv 생성 (훈련과정 중 어느 Epoch	시점에 테스트를 수행하여 submission.csv 를 구성해야 하는지 고찰하기)
가장 낮은 검증 손실(Validation Loss)을 기록한 Epoch의 모델 가중치를 저장하고 해당 시점의 모델로 submission.csv를 구성해야 한다. 이유는 바로 모델의 일반화 성능을 극대화하기 위함이다.

1. 최적 Epoch 선택의 이유: 과적합(Overfitting) 방지
딥러닝 모델의 훈련 과정은 보통 다음 세 단계를 거친다.
A. 훈련 손실 (Training Loss)
- 모델이 훈련 데이터를 얼마나 잘 예측하는지 나타낸다.
- 훈련을 계속할수록 일반적으로 계속 감소한다.
B. 검증 손실 (Validation Loss)
- 모델이 처음 보는 데이터 (검증 데이터)를 얼마나 잘 예측하는지 나타낸다.
이 손실이 모델의 일반화(Generalization) 성능을 측정하는 핵심 지표이다.
C. 과적합 (Overfitting)의 발생
훈련 초기에는 훈련 손실과 검증 손실이 모두 감소하며 모델 성능이 향상된. 그러나 훈련이 너무 오래 지속되면:
- 훈련 손실: 계속 감소한다.
- 검증 손실: 어느 시점(최적 Epoch) 이후에 다시 증가하기 시작한다.
검증 손실이 다시 증가하는 시점은 모델이 훈련 데이터의 노이즈나 사소한 특징까지 외우기 시작했다는 의미이며, 이것이 바로 과적합(Overfitting)이다.

2. 결론: 최적의 테스트 시점
따라서, 테스트(Test)는 모델의 일반화 성능이 가장 좋았던 시점, 즉 검증 손실이 최솟값이었던 Epoch에서 수행해야 한다.
시점	       훈련 손실	     검증 손실	     모델 상태	                   테스트 적합성
초기            높음	         높음	         과소적합 (Underfitting)	       ❌
최적 Epoch	    낮음	     최저	         최적의 일반화	               ✅ (이때 가중치를 저장해야 함)
후기	        매우 낮음	 증가	         과적합 (Overfitting)	       ❌
여기서는 400 Epoch에서 검증 손실이 가장 낮기 때문에 모델의 가중치는 400 Epoch 시점에 저장했으며, 최종 submission.csv 파일은 이 Epoch 400의 가중치를 로드한 모델로 예측을 수행하여 구성했다.

In [8]:
def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3, 
    'n_hidden_unit_list': [20, 20],
  }

  wandb.init(
    mode="online" if args.wandb else "disabled", # wandb 사용 여부 설정
    project="Titanic_ML_Competition", # 프로젝트 이름 설정
    notes="My first wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config # 하이퍼파라미터를 wandb에 로깅
  )
  print(args) # 출력 > Namespace(wandb=True, batch_size=128, epochs=400)
  print(wandb.config) # 출력 > {'epochs': 400, 'batch_size': 128, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}

  train_data_loader, validation_data_loader, test_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  # 훈련 시작
  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )

  generate_submission_csv(
    model=linear_model, 
    test_data_loader=test_data_loader, 
    base_path=BASE_PATH
  )
  wandb.finish() # wandb 실행 완료


if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False"
  )

  # [요구사항 2] : 128이 검증 손실 측면에서 가장 안정적이고 낮은 성능을 보였으므로 선택됨
  parser.add_argument(
    "-b", "--batch_size", type=int, default=128, help="Batch size (int, default:128)"
  )

  # [요구사항 3] : 400 Epoch에서 검증 손실이 가장 낮기 때문에 모델의 가중치는 400 Epoch 시점에 저장했으며, 최종 submission.csv 파일은 이 Epoch 400의 가중치를 로드한 모델로 예측을 수행하여 구성
  parser.add_argument(
    "-e", "--epochs", type=int, default=400, help="Number of training epochs (int, default:400)"
  )

  args = parser.parse_args('') # Jupyter Notebook 환경에서 인자를 파싱하기 위해 빈 문자열을 전달

  main(args)

wandb: Currently logged in as: seoeun2866 (seoeun2866-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Namespace(wandb=True, batch_size=128, epochs=400)
{'epochs': 400, 'batch_size': 128, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1

Epoch,▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇█████
Training loss,█▇▇▇▆▆▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▂▁▁▁▁▁▁
Validation loss,█▇▇▇▆▅▅▅▅▅▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Epoch,400
Training loss,0.38059
Validation loss,0.42145


### [요구사항 2] Batch Size 변경 및 선택

2.세 번째와 네 번째 차트(Sigmoid 모델의 배치 크기 비교/wandb.pdf 파일에 있음)를 분석 (이 차트들은 1절에서 가장 좋다고 판단된 Sigmoid 모델의 배치 크기(16, 32, 64, 128)별 성능을 보여줌)

2.1. 훈련 손실 (Training Loss) 분석

Sigmoid_16 (연두색 선)이 가장 낮은 훈련 손실(≈ 0.3)을 달성하여 훈련 데이터에 대한 적합도가 가장 높다. Sigmoid_32 (파란색 선), Sigmoid_64 (청록색 선), Sigmoid_128 (빨간색 선)은 비슷한 수준의 훈련 손실(≈ 0.33 ~ 0.35)을 보인다.

2.2. 검증 손실 (Validation Loss) 분석

Sigmoid_128 (빨간색 선)이 가장 낮은 검증 손실(≈ 0.4 ~ 0.43)을 보이며, 가장 안정적으로 수렴한다. 이는 일반화 성능이 가장 좋음을 나타낸다. Sigmoid_32 (파란색 선)와 Sigmoid_64 (청록색 선)는 Sigmoid_128보다 약간 높은 손실(≈ 0.43 ~ 0.48)을 보인다. Sigmoid_16 (연두색 선)은 훈련 손실은 가장 낮았으나, 검증 손실은 가장 높고 시간이 지날수록 발산하는 경향(≈ 0.7)을 보인다. 이는 과적합(Overfitting)의 전형적인 징후이다.

2.3. 결론: 최적의 배치 크기 128일 때 검증 손실이 가장 낮고 안정적이므로, 가장 좋은 일반화 성능을 보인다. 따라서 여기서는 배치 크기 128을 선택했다.

###  <자신이 취득한 기술적 사항/고찰 내용>

가. Sigmoid 선택의 고찰
- ReLU 계열과의 비교: 딥러닝에서 가장 인기 있는 ReLU는 기울기 소실(Vanishing Gradient) 문제에 강하고, 계산 효율적이며, 네트워크에 희소성(Sparsity)을 부여한다는 장점이 있다.
- 실습 결과 기반 선택: 그러나 타이타닉 이진 분류 실습 결과, Sigmoid 활성화 함수를 사용한 모델이 ReLU, ELU, Leaky ReLU보다 가장 낮은 검증 손실을 보이며 최고의 일반화 성능을 달성했다. 이는 데이터셋의 특성이나 모델의 깊이(얕은 신경망)에 따라 전통적인 Sigmoid가 여전히 우수한 성능을 보일 수 있음을 시사한다.
  
나. 최적 Epoch의 중요성 (Early Stopping)]
- 모델이 훈련 데이터를 완벽하게 학습하더라도, 처음 보는 데이터(검증 데이터)에 대한 성능이 저하되는 과적합(Overfitting) 현상이 발생할 수 있다.
- 기술적 판단: 따라서, 모델의 성능을 가장 잘 대표하는 지표는 검증 손실(Validation Loss)이며, 검증 손실이 최솟값을 기록하는 Epoch 시점의 모델 가중치를 최적 가중치로 저장하고 테스트에 사용해야 최대의 일반화 성능을 보장한다.

다. Adam Optimizer의 역할: 실습에서는 SGD의 진동 문제와 달리, 적응적 학습률을 제공하는 Adam을 사용하여 미니 배치에서 발생하는 진동을 효과적으로 제어하며 빠르게 수렴하도록 유도했다.

### [요구사항 - 4] submission.csv 제출 및 등수확인

![요구사항 - 4submission.csv제출 및 등수확인](https://raw.githubusercontent.com/REDMOONsupernova/image/refs/heads/main/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202025-10-16%20214441.png)

## **숙제 후기**

이번 숙제는 강의 시간에 배웠던 딥러닝의 핵심 개념들이 실제로 코딩을 통해 어떻게 구현되고 동작하는지 체감할 수 있었던 과제였다.
처음에 Wandb 로 그래프를 생성해서 제출하라고 하시길래 어떻게 생성하지 라는 막막함이 있었다. 'Wandb'라는 단어 자체도 생소했던 것 같다. 하지만 Wandb를 사용하니 여러 하이퍼파라미터 조건에서의 훈련 결과를 한눈에 그래프로 비교할 수 있어, 어떤 조건이 가장 효율적이었는지 빠르고 객관적으로 판단할 수 있어서 도움이 많이 되었다. 다른 과제에서도 Wandb를 활용한다면 더 효율적으로 할 수 있겠다는 생각이 들었다. 모델을 다르게 하거나 batch 사이즈를 다르게 해서 실험하는 과정에서 프로그램을 실행시킬 때마다 값이 달라져서 조금 애를 먹기는 했지만 직접 해봄으로서 의의 있었던 경험이었다. 다만 조금 아쉬웠던 것은 훈련 손실(Training Loss)은 계속 떨어지는데, 검증 손실(Validation Loss)은 다시 올라가는 지점(과적합)을 봤을 때였다. 이 경험을 통해 모델이 훈련 데이터만 외우는 것을 멈추고, 새로운 데이터에 대해서도 잘 작동하는 시점을 찾는 것이 얼마나 중요한지 깨달았다. 결국, 가장 낮은 검증 손실을 기록한 Epoch의 모델을 선택하는 것이 딥러닝 모델 개발의 핵심이라는 것을 이 과정을 통해서 알게되었다. 또한 수업을 통해 얻어가는 지식과 달리 이번 숙제를 통해 직접 해보며 딥러닝의 과정을 알아갈 수 있었다. 다음에는 이것보다 조금 더 복잡한 모델을 다루고 싶다는 생각을 하게 되었다.